In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace 'your_dataset.csv' with the actual dataset file)
data = pd.read_csv('preprocessed_data.csv')

# Assuming the last column is the target variable (y), and the rest are features (X)
X = data.drop('y', axis=1)  # Assuming 'y' is your target column
y = data['y']

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Example: Check the shape of the datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (36168, 42)
X_test shape: (9043, 42)


In [45]:
import joblib
from sklearn.metrics import accuracy_score

# Load the saved model
rf_model = joblib.load('random_forest_model.pkl')

# Predict on the test set
rf_predictions = rf_model.predict(X_test)

# Calculate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy :.4f}")


Random Forest Accuracy: 0.8865


In [ ]:
# import tensorflow as tf
# from sklearn.metrics import accuracy_score

# # Load the saved TensorFlow model
# tensorflow_model = tf.keras.models.load_model('best_tuned_mode_Ten.h5')

# # Predict on the test set
# tensorflow_predictions = tensorflow_model.predict(X_test).argmax(axis=1)

# # Calculate accuracy
# tensorflow_accuracy = accuracy_score(y_test, tensorflow_predictions)
# print(f"TensorFlow NN Accuracy: {tensorflow_accuracy * 100:.2f}%")


In [40]:
import torch

# Load the saved model state dict
model = torch.load("best_NN_model.pth", map_location='cpu')

# Check the layers in the model state dict
for name, param in model.items():
    print(f"{name}: {param.shape}")


fc1.weight: torch.Size([32, 42])
fc1.bias: torch.Size([32])
fc2.weight: torch.Size([2, 32])
fc2.bias: torch.Size([2])


C:\Users\wasif\AppData\Local\Temp\ipykernel_19656\2204771781.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("best_NN_model.pth", map_location='cpu')


In [51]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Define the model class (same as used during training)
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Function to load the model
def load_model(model_path, input_size, hidden_size, output_size, device):
    model = NeuralNet(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set model to evaluation mode
    return model

# Function to evaluate the model on a dataset (accuracy and classification report)
def evaluate_model_on_dataset(model, X_test, y_test, batch_size, device):
    # Convert the test dataset into a DataLoader
    test_dataset = TensorDataset(torch.tensor(X_test.values, dtype=torch.float32), 
                                  torch.tensor(y_test.values, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()  # Set model to evaluation mode
    y_pred, y_true = [], []
    with torch.no_grad():  # No gradients needed for evaluation
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU if available
            outputs = model(X_batch)  # Get model predictions
            _, predicted = torch.max(outputs, 1)  # Get the predicted class index
            y_pred.extend(predicted.cpu().numpy())  # Move predictions to CPU
            y_true.extend(y_batch.cpu().numpy())  # Move true labels to CPU
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Print accuracy
    print(f"Accuracy: {accuracy:.4f}")
    
    # Print classification report (precision, recall, F1 score)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

# Load your test dataset (X_test, y_test)
# Assuming X_test and y_test are pandas DataFrames or NumPy arrays
# For example:
# X_test = pd.read_csv("X_test.csv")  # or use the dataset you have
# y_test = pd.read_csv("y_test.csv")  # or use the dataset you have

# Define your model's input, hidden, and output sizes
input_size = X_test.shape[1]  # Assuming X_test is a DataFrame
hidden_size = 32  # You can change this to the value used during training
output_size = 2  # Assuming binary classification

# Path to the saved model
model_path = "best_NN_model.pth"

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model = load_model(model_path, input_size, hidden_size, output_size, device)

# Evaluate the model on the test dataset
batch_size = 32  # Adjust the batch size based on your memory
evaluate_model_on_dataset(model, X_test, y_test, batch_size, device)


C:\Users\wasif\AppData\Local\Temp\ipykernel_19656\3095676625.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Accuracy: 0.9074

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7952
           1       0.66      0.49      0.56      1091

    accuracy                           0.91      9043
   macro avg       0.79      0.73      0.75      9043
weighted avg       0.90      0.91      0.90      9043

